In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sc1005-news/True.csv
/kaggle/input/sc1005-news/Fake.csv


In [2]:
true = pd.read_csv('/kaggle/input/sc1005-news/True.csv')
false = pd.read_csv('/kaggle/input/sc1005-news/Fake.csv')

In [3]:
# Add a 'truth' column to each dataframe with the appropriate values
true['label'] = 1
false['label'] = 0

# for presentation 
true['label_meaning'] = True 
false['label_meaning'] = False

# Concatenate the two dataframes into a single dataframe
combined_news = pd.concat([true, false], ignore_index=True)

# shuffle the rows
news = combined_news.sample(frac=1).reset_index(drop=True)
news.head()

,title,text,subject,date,label,label_meaning
0,Trump's immigration pitch falls flat with Repu...,(Reuters) - Republican presidential candidate ...,politicsNews,"September 1, 2016",1,True
1,Germany seeks to maintain unity if U.S. decert...,BERLIN (Reuters) - The German government on Fr...,worldnews,"October 13, 2017",1,True
2,North Korea sentences South Korean reporters t...,SEOUL (Reuters) - A North Korean court sentenc...,worldnews,"August 31, 2017",1,True
3,Sanders hits Clinton on campaign finance hours...,NEW YORK (Reuters) - U.S. Democratic president...,politicsNews,"April 18, 2016",1,True
4,"Incensed over refugees, east Germans punish ea...","BERLIN (Reuters) - For weeks, Chancellor Angel...",worldnews,"September 24, 2017",1,True


In [4]:
#create new dataframe with title,text,label
news_cleaned = news[['title','text','label']]

In [5]:
from sklearn.utils import shuffle

# Purify by merging the title and text
news_cleaned['comb_text'] = news_cleaned[['title', 'text']].agg(''.join, axis=1)
display(news_cleaned)
# Shuffle
news_cleaned = shuffle(news_cleaned).reset_index(drop=True)


/tmp/ipykernel_23/3886359515.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_cleaned['comb_text'] = news_cleaned[['title', 'text']].agg(''.join, axis=1)


,title,text,label,comb_text
0,Trump's immigration pitch falls flat with Repu...,(Reuters) - Republican presidential candidate ...,1,Trump's immigration pitch falls flat with Repu...
1,Germany seeks to maintain unity if U.S. decert...,BERLIN (Reuters) - The German government on Fr...,1,Germany seeks to maintain unity if U.S. decert...
2,North Korea sentences South Korean reporters t...,SEOUL (Reuters) - A North Korean court sentenc...,1,North Korea sentences South Korean reporters t...
3,Sanders hits Clinton on campaign finance hours...,NEW YORK (Reuters) - U.S. Democratic president...,1,Sanders hits Clinton on campaign finance hours...
4,"Incensed over refugees, east Germans punish ea...","BERLIN (Reuters) - For weeks, Chancellor Angel...",1,"Incensed over refugees, east Germans punish ea..."
...,...,...,...,...
44893,He ‘Belongs In An Institution’: James Comey’s...,Amateur president Donald Trump s repeated atta...,0,He ‘Belongs In An Institution’: James Comey’s...
44894,MONEY TO “BERN!”…SOCIALIST BERNIE SANDERS Endo...,Feel the Bern of a lot of gullible young peopl...,0,MONEY TO “BERN!”…SOCIALIST BERNIE SANDERS Endo...
44895,"Once Again, Trump Proves How INCREDIBLY Ignor...",President Donald Trump does not know anything ...,0,"Once Again, Trump Proves How INCREDIBLY Ignor..."
44896,Factbox: What it will take for Trump to meet h...,WASHINGTON (Reuters) - President-elect Donald ...,1,Factbox: What it will take for Trump to meet h...


In [6]:
news_cleaned = news_cleaned[['comb_text','label']]

In [7]:
df_texts, df_labels = news_cleaned["comb_text"].to_list(),news_cleaned["label"].to_list()

In [8]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(df_texts, df_labels, test_size=.2)

In [9]:
from transformers import BertTokenizer, TFBertModel,BertForSequenceClassification,DistilBertTokenizerFast,DistilBertModel

2024-04-16 16:01:43.400167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 16:01:43.400298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 16:01:43.532707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
model_name = DistilBertModel.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [12]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [14]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batc

In [15]:
trainer.train()

Step,Training Loss
10,0.698400
20,0.692700
30,0.683900
40,0.665700
50,0.643500
60,0.591200
70,0.512000
80,0.427700
90,0.310200
100,0.214800


TrainOutput(global_step=6735, training_loss=0.011700694829895188, metrics={'train_runtime': 2922.2635, 'train_samples_per_second': 36.873, 'train_steps_per_second': 2.305, 'total_flos': 1.4273892074778624e+16, 'train_loss': 0.011700694829895188, 'epoch': 3.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.00037280391552485526,
 'eval_accuracy': 0.9998886414253898,
 'eval_f1': 0.9998833274997083,
 'eval_precision': 1.0,
 'eval_recall': 0.9997666822211853,
 'eval_runtime': 81.6001,
 'eval_samples_per_second': 110.049,
 'eval_steps_per_second': 1.728,
 'epoch': 3.0}